# Introduction

Author: Yingding Wang\
Created on: 03.01.2024

In [1]:
import sys

In [2]:
from platform import python_version
print (f"current platform python version: {python_version()}")

current platform python version: 3.11.6


In [3]:
!{sys.executable} -m pip install --upgrade --user kfp==2.4.0 kfp-kubernetes==1.0.0 kfp-server-api==2.0.5

  Using cached kfp-2.4.0-py3-none-any.whl
  Using cached kfp_kubernetes-1.0.0-py3-none-any.whl
  Using cached kfp_server_api-2.0.5-py3-none-any.whl
  Using cached kfp_pipeline_spec-0.2.2-py3-none-any.whl (20 kB)
  Attempting uninstall: kfp-pipeline-spec
    Found existing installation: kfp-pipeline-spec 0.1.16
    Uninstalling kfp-pipeline-spec-0.1.16:
      Successfully uninstalled kfp-pipeline-spec-0.1.16
  Attempting uninstall: kfp-server-api
    Found existing installation: kfp-server-api 1.8.5
    Uninstalling kfp-server-api-1.8.5:
      Successfully uninstalled kfp-server-api-1.8.5
  Attempting uninstall: kfp
    Found existing installation: kfp 1.8.22
    Uninstalling kfp-1.8.22:
      Successfully uninstalled kfp-1.8.22
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [4]:
!{sys.executable} -m pip list | grep kfp

kfp                           2.4.0
kfp-kubernetes                1.0.0
kfp-pipeline-spec             0.2.2
kfp-server-api                2.0.5


In [5]:
from kfp.dsl.pipeline_task import PipelineTask

def set_res_limit(task: PipelineTask, mem_req="200Mi", cpu_req="2000m", mem_lim="4000Mi", cpu_lim='4000m') -> PipelineTask:
    """set the resource limit for cpu and memory, no cpu and memory requirement sofar.
    should the limit is set to small, the Task Pod would be stopped by kubernetes with OOMKilled status.
    
    Args:
        task(PipelineTask): the KFP PipelineTask which need to be set the cpu and memory limits
        cpu_limit(str): the str representation of cpu limit e.g. '1' as one cpu time, '0.5' as 1/2 cpu time
        mem_limit(str): the str representation of memory limit e.g. '500M' for 500MB RAM
        
    Return:
        (PipelineTask): the PipelineTask with the desired limitations set
    """
    # return task.set_cpu_limit('1').set_memory_limit('500M')
    return task.set_cpu_request(cpu_req)\
            .set_cpu_limit(cpu_lim)\
            .set_memory_request(mem_req)\
            .set_memory_limit(mem_lim)

In [6]:
from kfp.client import Client
from kfp import dsl
import kfp


@dsl.component(
    base_image='python:3.10.13-bullseye'    
)
def add(a: float, b: float) -> float:
    '''Calculates sum of two arguments'''
    return a + b


@dsl.pipeline(
    name='Addition pipeline',
    description='An example pipeline that performs addition calculations.')
def add_pipeline(
    a: float = 1.0,
    b: float = 7.0,
):
    first_add_task = add(a=a, b=4.0)
    first_add_task = set_res_limit(task=first_add_task, cpu_req='500m', mem_req='200M')
    second_add_task = add(a=first_add_task.output, b=b)
    second_add_task = set_res_limit(task=second_add_task, cpu_req='500m', mem_req='200M')

    
client = Client()
NAMESPACE = client.get_user_namespace()
EXPERIMENT_NAME = "demo"
print(NAMESPACE)

kubeflow-kindfor


/home/jovyan/.local/lib/python3.11/site-packages/kfp/client/client.py:159: FutureWarning: This client only works with Kubeflow Pipeline v2.0.0-beta.2 and later versions.
  warnings.warn(


In [7]:
# ENABLE_CACHING = False
ENABLE_CACHING = True

client.create_run_from_pipeline_func(
    pipeline_func = add_pipeline, 
    experiment_name = EXPERIMENT_NAME,
    namespace = NAMESPACE,
    enable_caching=ENABLE_CACHING,
    # mode=kfp.dsl.PipelineExecutionMode.V2_COMPATIBLE,
    arguments={
        'a': 7.0,
        'b': 8.0
    })

RunPipelineResult(run_id=37a7ddbc-fe2e-4eb5-bdbb-30f29275b8a4)